In [2]:
import os.path as osp
import time
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid, WebKB, HeterophilousGraphDataset, TUDataset
from torch_geometric.nn import GATConv
from torch_geometric.utils import to_networkx, from_networkx, to_dense_adj

import networkx as nx

from FormanRicci import FormanRicci
from attention_curvature import Experiment, GAT, compute_edge_curvatures

mutag = list(TUDataset('data', name='MUTAG'))

Extracting data/MUTAG/MUTAG.zip
Processing...
Done!


In [10]:
num_features = mutag[0].num_features
num_classes = 2
num_layers = 1
num_heads = 1

model = GAT(num_features, num_classes, num_layers, num_heads)

In [3]:
mutag_1_layer = Experiment(mutag, 1)

AttributeError: 'GlobalStorage' object has no attribute 'num_classes'